# **3) VLM Readers & Labels**

In [1]:
from pathlib import Path
import os, sys, platform, json, subprocess

ROOT = Path.cwd().resolve().parents[0]

if (ROOT / "src").exists():
    sys.path.insert(0, str(ROOT))

print("Project root:", ROOT)
print("Python:", sys.version)
print("OS:", platform.platform())

from src.config.loader import load_cfg
cfg = load_cfg()
cfg.labels

Project root: D:\IIT BBS\Job Resources\fiolabs\diagram-intel
Python: 3.11.13 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:03:15) [MSC v.1929 64 bit (AMD64)]
OS: Windows-10-10.0.26100-SP0


{'use_vlm_on_micro': False, 'max_tiles_vlm': 120, 'min_vec_chars': 2, 'merge_dedup_fuzz': 88}

#### **Building vector text index**

In [2]:
from src.vision.runners.labels_reader import build_vector_text_index
vec_idx = build_vector_text_index(cfg)
vec_idx

2025-08-27 04:01:53.448 | INFO     | src.vision.runners.labels_reader:build_vector_text_index:107 - [vector_text] wiring diagram_Main Distribution Panel_Fio Labs: 260 items across 1 pages (sources: ['pymupdf'])


{'wiring diagram_Main Distribution Panel_Fio Labs': [{'page': 1,
   'path': 'D:\\IIT BBS\\Job Resources\\fiolabs\\diagram-intel\\data\\processed\\vector_text\\wiring diagram_Main Distribution Panel_Fio Labs\\page-1.json',
   'count': 260,
   'source': 'pymupdf'}]}

#### **Tile label mapping (vector only first)**

In [3]:
from src.vision.runners.labels_reader import run_labels_reader
run_labels_reader(cfg)

2025-08-27 04:01:57.881 | INFO     | src.vision.runners.labels_reader:run_labels_reader:138 - [labels] VLM disabled (vector-only).


2025-08-27 04:01:58.335 | INFO     | src.vision.runners.labels_reader:run_labels_reader:211 - [labels] wrote 450 tile label files → D:\IIT BBS\Job Resources\fiolabs\diagram-intel\data\processed\labels\tile_labels.index.json


#### **Peek**

In [4]:
from src.utils.io import read_json
from pathlib import Path
idx = read_json(Path(cfg.paths.processed)/"labels"/"tile_labels.index.json")
len(idx), idx[:3]

(450,
 [{'tile_json': 'D:\\IIT BBS\\Job Resources\\fiolabs\\diagram-intel\\data\\processed\\labels\\tiles\\wiring diagram_Main Distribution Panel_Fio Labs\\page-1\\tile_r000_c000.json',
   'n_vec': 0,
   'n_vlm': 0},
  {'tile_json': 'D:\\IIT BBS\\Job Resources\\fiolabs\\diagram-intel\\data\\processed\\labels\\tiles\\wiring diagram_Main Distribution Panel_Fio Labs\\page-1\\tile_r000_c001.json',
   'n_vec': 0,
   'n_vlm': 0},
  {'tile_json': 'D:\\IIT BBS\\Job Resources\\fiolabs\\diagram-intel\\data\\processed\\labels\\tiles\\wiring diagram_Main Distribution Panel_Fio Labs\\page-1\\tile_r000_c002.json',
   'n_vec': 3,
   'n_vlm': 0}])

In [7]:
from src.utils.io import read_json
from pathlib import Path

# find a tile with some labels
tiles_dir = Path(cfg.paths.processed)/"labels"/"tiles"
candidates = list(tiles_dir.rglob("*.json"))
print("Tile JSON files:", len(candidates))
sample = next(p for p in candidates if p.stat().st_size > 1000)  # heuristic
sample, read_json(sample)


Tile JSON files: 450


(WindowsPath('D:/IIT BBS/Job Resources/fiolabs/diagram-intel/data/processed/labels/tiles/wiring diagram_Main Distribution Panel_Fio Labs/page-1/tile_r000_c002.json'),
 {'pdf': 'wiring diagram_Main Distribution Panel_Fio Labs',
  'page': 1,
  'scale': 'micro',
  'row': 0,
  'col': 2,
  'tile_bbox': [870, 0, 1382, 512],
  'tile_path': 'D:\\IIT BBS\\Job Resources\\fiolabs\\diagram-intel\\data\\interim\\tiles\\micro\\wiring diagram_Main Distribution Panel_Fio Labs\\page-1\\tile_r000_c002.png',
  'vector_labels': ['To', 'Inverter', 'TB1-4'],
  'vlm_labels': [],
  'labels_merged': ['To', 'Inverter', 'TB1-4'],
  'vector_items': [{'text': 'To',
    'bbox': [1309.49, 118.28, 1402.18, 203.65],
    'x': 1309.49,
    'y': 118.28},
   {'text': 'Inverter',
    'bbox': [1186.49, 188.77, 1478.25, 274.14],
    'x': 1186.49,
    'y': 188.77},
   {'text': 'TB1-4',
    'bbox': [1312.23, 458.69, 1397.6, 678.0],
    'x': 1312.23,
    'y': 458.69}]})